In [91]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [92]:
from pyathena import connect
import math
import pandas as pd

In [93]:
conn = connect(s3_staging_dir='s3://hyperflow-athena-results',
               region_name='us-east-1')

In [94]:
METRICS_COLS = ','.join(['time', 'workflowId', 'jobId', 'name', 'parameter', 'valueStr', 'valueFloat', 'valueInt', 'valueDict'])
SYS_INFO_COLS = ','.join(['cpu', 'mem', 'jobId'])
JOB_DESCRIPTION_COLS = ','.join(['workflowName', 'size', 'version', 'hyperflowId', 'jobId', 'env', 'nodeName', 'executable', 'args', 'inputs', 'outputs', 'name', 'command'])

In [95]:
def query_athena(table, columns):
    return pd.read_sql('SELECT {} FROM hyperflow_logs.{};'.format(columns, table), conn)

def merge_value_num(df):
    df['valueNum'] = [val[0] if math.isnan(val[1]) else val[1] for val in zip(df['valueFloat'], df['valueInt'])]

In [96]:
metrics_df = query_athena('montage2__0_01__1_0_0__2020_06_01_18_14_25', METRICS_COLS) # aws/t3.large/montage2__0_01__1_0_0__2020_06_01_18_14_25
metrics_df = metrics_df[metrics_df['time'].notnull()]
merge_value_num(metrics_df)
metrics_df

,time,workflowId,jobId,name,parameter,valueStr,valueFloat,valueInt,valueDict,valueNum
31,2020-06-01T18:13:39.781,-tDswEFbA-1,-tDswEFbA-1-12,mProject,event,handlerStart,NaN,NaN,None,NaN
32,2020-06-01T18:13:39.828,-tDswEFbA-1,-tDswEFbA-1-12,mProject,io,None,NaN,NaN,"{read=135374, write=1, readsyscalls=12, writes...",NaN
33,2020-06-01T18:13:39.829,-tDswEFbA-1,-tDswEFbA-1-12,mProject,network,None,NaN,NaN,"{read=null, write=null, readsyscalls=null, wri...",NaN
34,2020-06-01T18:13:39.844,-tDswEFbA-1,-tDswEFbA-1-12,mProject,event,jobStart,NaN,NaN,None,NaN
35,2020-06-01T18:13:39.909,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,NaN,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...
590,2020-06-01T18:14:21.699,-tDswEFbA-1,-tDswEFbA-1-31,mViewer,cpu,None,NaN,0.0,None,0.0
591,2020-06-01T18:14:21.699,-tDswEFbA-1,-tDswEFbA-1-31,mViewer,memory,None,NaN,7983104.0,None,7983104.0
592,2020-06-01T18:14:21.699,-tDswEFbA-1,-tDswEFbA-1-31,mViewer,ctime,None,NaN,60.0,None,60.0
593,2020-06-01T18:14:22.007,-tDswEFbA-1,-tDswEFbA-1-31,mViewer,event,jobEnd,NaN,NaN,None,NaN


In [97]:
metrics_df[(metrics_df['parameter']=='memory') & (metrics_df['valueNum'] > 0)].groupby('name')['valueNum'].mean()

name
mAdd           1.411072e+07
mBackground    2.469615e+07
mBgModel       1.373116e+07
mConcatFit     1.733564e+07
mDiffFit       4.167680e+06
mImgtbl        1.406566e+07
mProject       1.142354e+07
mViewer        7.494656e+06
Name: valueNum, dtype: float64

In [98]:
cpu_metrics_df = metrics_df[(metrics_df['parameter']=='cpu') & (~metrics_df['valueNum'].isna()) & (metrics_df['valueNum'] > 0)]
cpu_metrics_df

,time,workflowId,jobId,name,parameter,valueStr,valueFloat,valueInt,valueDict,valueNum
40,2020-06-01T18:13:41.916,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,96.551724,NaN,None,96.551724
45,2020-06-01T18:13:43.917,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,94.500000,NaN,None,94.500000
50,2020-06-01T18:13:45.917,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,95.500000,NaN,None,95.500000
55,2020-06-01T18:13:47.919,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,95.522388,NaN,None,95.522388
60,2020-06-01T18:13:49.923,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,NaN,94.0,None,94.000000
65,2020-06-01T18:13:51.927,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,96.500000,NaN,None,96.500000
70,2020-06-01T18:13:53.934,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,94.029851,NaN,None,94.029851
75,2020-06-01T18:13:55.942,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,95.522388,NaN,None,95.522388
80,2020-06-01T18:13:57.962,-tDswEFbA-1,-tDswEFbA-1-12,mProject,cpu,None,94.527363,NaN,None,94.527363
94,2020-06-01T18:13:41.915,-tDswEFbA-1,-tDswEFbA-1-1,mProject,cpu,None,96.517413,NaN,None,96.517413


In [99]:
metrics_df[metrics_df['parameter']=='ctime'].groupby('name')['valueNum'].mean()

name
mAdd             30.000000
mBackground      51.666667
mBgModel         16.666667
mConcatFit       33.333333
mDiffFit         25.000000
mImgtbl          26.666667
mProject       8585.000000
mViewer          30.000000
Name: valueNum, dtype: float64